In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [ ]:
import pathlib
fire_image_path_array = ["../input/forest-fire-images/Data/Train_Data/Fire","/kaggle/input/forest-fire-image-dataset/FOREST_FIRE_DATASET/FIRE","/kaggle/input/fire-dataset/fire_dataset/fire_images","/kaggle/input/forest-fire-images/Data/Test_Data/Fire","/kaggle/input/test-dataset/Fire-Detection/1","/kaggle/input/fire-and-smoke-dataset/Datacluster Fire and Smoke Sample"]
non_fire_path_array = ["../input/forest-fire-images/Data/Train_Data/Non_Fire","/kaggle/input/forest-fire-image-dataset/FOREST_FIRE_DATASET/NON_FIRE","/kaggle/input/fire-dataset/fire_dataset/non_fire_images","/kaggle/input/forest-fire-images/Data/Test_Data/Non_Fire","/kaggle/input/test-dataset/Fire-Detection/0"]
fire_image_path = pathlib.Path(fire_image_path_array[0])
non_fire_path = pathlib.Path(non_fire_path_array[0])
for path in fire_image_path_array[1:]:
    fire_image_path.joinpath(path)
    
for path in non_fire_path_array[1:]:
    non_fire_path.joinpath(path)

In [ ]:
import glob
from itertools import chain

train_data_images = {
    "Fire":list(chain(fire_image_path.glob('*.jpg'), fire_image_path.glob('*.png'))),
    "NonFire":list(chain(non_fire_path.glob('*.jpg'), non_fire_path.glob('*.png')))
}
train_labels = {
    "Fire":0,"NonFire":1
}

In [ ]:
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn 
import tensorflow as tf
from tensorflow import keras
%matplotlib inline

In [ ]:
X, y = [], []
for label, images in train_data_images.items():
    for image in images:
        img = cv2.imread(str(image)) # Reading the image
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (180, 180))
            X.append(img)
            y.append(train_labels[label])

In [ ]:
import numpy
X_samp = numpy.array(X)
y_samp = numpy.array(y)

In [ ]:
X_samp = (X_samp/255)

In [ ]:
X_samp.shape


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_samp, y_samp, test_size=0.2)

In [ ]:
data_argumentation = keras.Sequential([
    keras.layers.experimental.preprocessing.RandomContrast(0.2),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    keras.layers.experimental.preprocessing.RandomZoom(0.2),
]

In [ ]:
plt.imshow(X_train[0])

In [ ]:
for i in range(8):
    plt.imshow(data_argumentation(X_train[0]))
    plt.show()

In [ ]:
model = keras.Sequential([
    data_argumentation,
    keras.layers.Conv2D(64, (3,3), padding='same', activation="relu", input_shape=(180, 180, 3)),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(32, (3,3), padding='same', activation="softmax"),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(16, (3,3), padding='same', activation="softmax"),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(10, activation="sigmoid"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=["accuracy"])


In [ ]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))